In [50]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## 📥 Load and Prepare the Dataset

In [51]:
data = load_breast_cancer()
X, y = data.data, data.target

### Feature Scaling

In [52]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#### Train-Test Split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## ⚙️ Building a Custom Logistic Regression Model

#### Steps to Train the Model

1. **Initialize parameters**  
   Set weights `w` and bias `b` to zero.

2. 🔁 **Repeat for a number of iterations**  
   For each iteration:
   
   - **Make predictions** using the sigmoid function:  
     `y_pred = sigmoid(X · w + b)`
   
   - ❌ **Calculate error**  
     `error = y_pred - y` (difference between prediction and actual labels)
   
   - **Compute gradients (with L2 regularization => to prevent overfitting )**  
     - `dw = (1/m) * (Xᵗ · error) + (λ/m) * w`  
     - `db = (1/m) * sum(error)`
   
   - 🛠️ **Update the weights and bias**  
     - `w = w - learning_rate × dw`  
     - `b = b - learning_rate × db`

3. ✅ **Make predictions after training**  
   - `predict_proba(X)` → returns probabilities  
   - `predict(X)` → returns 0 or 1 based on threshold (default 0.5)

In [54]:
class CustomLogisticRegression:
    def __init__(self, learning_rate=0.01, iterations=5000, lamda=0.1):
        self.learning_rate = learning_rate
        self.iterations = iterations
        self.lamda = lamda
        self.w = None
        self.b = None
        self.costs = []

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def compute_gradient_regularized(self, X, y, w, b, lamda):
        m = X.shape[0]
        z = X.dot(w) + b
        y_pred = self.sigmoid(z)
        err = y_pred - y
        dw = (1/m) * (X.T.dot(err)) + (lamda/m)*w
        db = (1/m) * np.sum(err)
        return dw, db

    def fit(self, X, y):
        m, n = X.shape
        self.w = np.zeros(n)
        self.b = 0.0

        for i in range(self.iterations):
            dw, db = self.compute_gradient_regularized(X, y, self.w, self.b, self.lamda)
            self.w -= self.learning_rate * dw
            self.b -= self.learning_rate * db

    def predict_proba(self, X):
        return self.sigmoid(X.dot(self.w) + self.b)

    def predict(self, X):
        return (self.predict_proba(X) >= 0.5).astype(int)

### 🏋️ Train Custom Model

In [55]:
custom_model = CustomLogisticRegression()
custom_model.fit(X_train, y_train)
y_pred_custom = custom_model.predict(X_test)

### 🤖 Train Sklearn Model

In [56]:
sklearn_clf = LogisticRegression()
sklearn_clf.fit(X_train, y_train)
y_pred_sk = sklearn_clf.predict(X_test)

### ✅ Models Evaluation

In [57]:
print("Custom Model Accuracy:  ", accuracy_score(y_test, y_pred_custom))
print("Sklearn Model Accuracy: ", accuracy_score(y_test, y_pred_sk))

Custom Model Accuracy:   0.9824561403508771
Sklearn Model Accuracy:  0.9736842105263158
